# LANGCHAIN 시작하기

## 1. LANGCHAIN 이란

언어 모델을 활용해 다양한 애플리케이션을 개발할 수 있는 프레임워크로 다음과 같은 기능을 수행 가능.

*   문맥 인식: LangChain은 다양한 언어 모델을 문맥 소스와 연결하며, 여기에는 프롬프트의 지시사항, 소수의 예시, 응답에 근거한 내용이 포함됨. 이를 통해 언어 모델은 제공된 정보를 기반으로 더 정확하고 관련성 높은 답변을 생성 가능.
*   추론: 언어 모델은 주어진 문맥을 바탕으로 답변을 제공하거나, 어떠한 조치를 취해야할지 스스로 추론. 이는 모델이 단순히 정보의 재생산을 넘어, 주어진 상황을 분석하고 적절한 해결책을 제시할 수 있음을 의미.



## 2. 설치

**LangChain 설치 및 업데이트**

In [ ]:
pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith

## 3. 프레임워크의 구성

*   라이브러리: Python 및 Javascript 라이브러리. 다양한 컴포넌트와 인터페이스와 통합, 여러 컴포넌트를 체인과 에이전트로 결합하는 기본 런타임. 그리고 즉시 사용 가능한 체인과 에이전트의 구현을 포함.
*   템플릿: 다양한 작업을 위한 쉽게 배포 가능한 참조 아키텍처
*   LangServe: LangChain을 REST API로 배포하기 위한 라이브러리
*   LangSmith: 어떠한 LLM 프레임워크에도 구축된 체인을 디버깅, 테스트, 평가, 모니터링 할 수 있게 해주며 LangChain과 원활하게 통합되는 개발자 플랫폼
*   LangGraph: LLM을 사용한 상태유지가 가능한 다중 액터 애플리케이션을 구축하기 위한 라이브러리



---



# Chat Open AI

**[속성]**


*   temperature: 샘플링 온도, 0에서 2사이의 값으로 선택, 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은  출력을 더 집중되고 결정론적으로 만듬.
*   max_tokens: 채팅 완성에 생성할 토큰의 최대 수
*   model_name: 적용할 모델 이름




In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1,
    max_tokens=2048,
    model_name="gpt-3.5-turbo"
)

question = "대한민국의 수도는 어디입니까?"

print(f"답변: {llm.invoke(question)}")


답변: content='대한민국의 수도는 서울입니다.' response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 25, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None} id='run-bf6b9d7c-6e98-4261-9cc0-5b4aba2a2077-0'




---



# PROMPT TEMPLATE

사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿

In [21]:
from langchain.prompts import PromptTemplate

template = "{country}의 수도는?"

prompt = PromptTemplate(template=template, input_variables=["country"])
print(f"prompt1: {prompt}")

print("-" * 20)

prompt2 = PromptTemplate.from_template(template=template)
print(f"prompt2: {prompt2}")

prompt1: input_variables=['country'] template='{country}의 수도는?'
--------------------
prompt2: input_variables=['country'] template='{country}의 수도는?'




---



# LLMChain

## 1. 기본 사용

LLMChain은 특정 PromptTemplate과 LLM이 연결된 체인 객체를 생성

[매개변수]
*    prompt: PromptTemplate
*    llm: 언어 모델

In [22]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1, max_tokens=2048, model_name="gpt-3.5-turbo")

template = "{country}의 수도는 어디입니까?"
prompt = PromptTemplate.from_template(template=template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

result = llm_chain.invoke({"country": "대한민국"})
print(f"답변: {result}")

답변: {'country': '대한민국', 'text': '대한민국의 수도는 서울입니다.'}


## 2. apply

apply() 함수를 사용하여 여러개의 입력에 대한 처리를 한 번에 수행 가능.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1, max_tokens=2048, model_name="gpt-3.5-turbo")

template = "{country}의 수도는 어디입니까?"
prompt = PromptTemplate.from_template(template=template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

# inputs
input_list = [
    {"country": "대한민국"},
    {"country": "미국"},
    {"country": "멕시코"},
    ]

# apply
results = llm_chain.apply(input_list)

for result in results:
  print(f"답변: {result}")


답변: {'text': '대한민국의 수도는 서울입니다.'}
답변: {'text': '미국의 수도는 워싱턴 D.C.입니다.'}
답변: {'text': '멕시코의 수도는 멕시코시티입니다.'}


## 3. generate

generate() 함수를 사용하면 LLMResult를 반환하며, 이 객체를 사용해 토큰의 사용량과 종료 이유 같은 부가적 정보를 확인 가능.
(llm_output를 확인하여 토큰 정보 등을 확인 가능)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1, max_tokens=2048, model_name="gpt-3.5-turbo")

template = "{country}의 수도는 어디입니까?"
prompt = PromptTemplate.from_template(template=template)

llm_chain = LLMChain(prompt=prompt, llm=llm)

# inputs
input_list = [
    {"country": "대한민국"},
    {"country": "미국"},
    {"country": "멕시코"},
    ]

result = llm_chain.generate(input_list)

print(f"RESULT: {result}")

RESULT: generations=[[ChatGeneration(text='대한민국의 수도는 서울입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 25, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-fbc2c9f3-d81c-4df0-82b1-c65af6fdf340-0'))], [ChatGeneration(text='미국의 수도는 워싱턴 D.C.입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='미국의 수도는 워싱턴 D.C.입니다.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 22, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-eea4104d-c2fa-491f-86d8-bacf08fad33f-0'))], [ChatGeneration(text='멕시코의 수도는 멕시코시티입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='멕시코의 수도는



---



# 스트리밍

## 1. 스트리밍

스트리밍 옵션을 사용하면 질의에 대한 응답을 실시간으로 받을 수 있음.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=0.1,
    max_tokens=2048,
    model_name="gpt-3.5-turbo",
    streaming=True, # 라잌디스
    callbacks=[StreamingStdOutCallbackHandler()]
)

question = "오스트랄로피테쿠스에 대해서 300자 이내로 설명해줘."

response = llm.invoke(question)

오스트랄로피테쿠스는 멸종된 인류의 조상으로 알려진 원시인류 종으로, 약 2백만 년 전부터 약 1백만 년 전까지 아프리카와 유라시아 대륙에서 서식했던 것으로 추정되고 있습니다. 이 종은 진화 과정에서 뇌 용량이 증가하고 양쪽 뇌 반구가 발달하는 등의 특징을 보였으며, 두 다리로 걷는 직립보행을 하였다는 점에서 현대인과 유사한 특징을 가지고 있습니다. 또한, 공격적인 특성을 가진 것으로 알려져 있으며, 뼈 속에 발견된 상처 흔적 등을 통해 그들의 생활 방식을 추론할 수 있습니다. 이 종은 현대인류의 진화 과정에서 중요한 역할을 한 것으로 여겨지며, 인류의 진화 역사를 연구하는데 중요한 자료로 활용되고 있습니다.



---



# LCEL(LangChain Expression Language)

## 1. LCEL 기본 설명

LCEL(LangChain Expression Language)를 사용하여 다양한 구성요소를 단일 체인으로 연결 가능

chain = prompt | model | output

기호는 | 이며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달.

In [15]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

template = "{country}의 수도는 어디입니까?"
prompt = PromptTemplate.from_template(template=template)

model = ChatOpenAI(model_name="gpt-3.5-turbo")

output_parser = StrOutputParser()

#Chain
chain = prompt | model | output_parser

result = chain.invoke({"country": "대한민국"})

print(f"답변: {result}")

답변: 대한민국의 수도는 서울 입니다.


## 2. LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록 "Runnable" 프로토콜이 구현되어 있으며, 이는 대부분의 컴포넌트에 구현되어 있음. 이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출는 것을 쉽게 만듬.


### 표준 인터페이스에 포함되는 것들

*   stream: 응답의 청크를 스트리밍
*   invoke: 입력에 대해 체인을 호출
*   batch: 입력 목록에 대해 체인을 호출

(비동기 인터페이스들도 존재하며, 표준 인터페이스 앞에 a를 붙임)

### stream: 실시간 출력


In [17]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

template = "{brand} 브랜드를 설명해줘."
prompt = PromptTemplate.from_template(template)

model = ChatOpenAI(model_name="gpt-3.5-turbo")

chain = prompt | model

for s in chain.stream({"brand": "Nike"}):
  print(s.content, end="", flush=True)


Nike는 전 세계적으로 유명한 스포츠 의류 및 운동화 브랜드로, 1971년에 빌 보워만과 필 나이트에 의해 설립되었습니다. Nike는 스포츠 선수들을 위한 혁신적인 제품과 기술을 개발하여 세계적으로 인기를 끌고 있습니다. 

Nike는 다양한 스포츠 분야에서 제품을 생산하며, 농구, 축구, 러닝, 골프, 테니스 등 다양한 스포츠 분야에서 활발하게 활동하고 있습니다. 또한 Nike는 다양한 스포츠 선수들과 협업하여 한정판 제품을 출시하는 등 브랜드의 이미지를 높이고 있습니다.

또한 Nike는 지속 가능한 제품 생산을 위해 노력하고 있으며, 다양한 사회 공헌 활동을 통해 사회적 책임을 다하고 있습니다. Nike는 전 세계적으로 많은 사람들에게 사랑받는 브랜드로 자리매김하고 있습니다.

stream 메서드를 사용하여 주어진 프롬프트의 답변에 대한 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용을 출력한다.

### batch: 배치

chain.batch 메서드는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 일괄 처리를 수행.
config의 max_concurrency 속성을 사용하여 동시에 처리할 수 있는 최대 작업 수를 설정 가능.

In [18]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

template = "{brand} 브랜드를 설명해줘."
prompt = PromptTemplate.from_template(template)

model = ChatOpenAI(model_name="gpt-3.5-turbo")

chain = prompt | model

result = chain.batch(
    [{"brand": "BMW"}, {"brand": "Audi"}, {"brand": "Kia"}],
	  config={"max_concurrency": 2}
)

print(f"결과: {result}")

결과: [AIMessage(content='BMW는 독일의 자동차 제조 업체로, Bayerische Motoren Werke AG의 약어입니다. 1916년에 설립된 BMW는 고급 자동차와 모터사이클을 생산하는 세계적인 럭셔리 자동차 브랜드로 널리 알려져 있습니다. BMW는 우수한 성능, 디자인, 기술력을 바탕으로 뛰어난 운전 경험을 제공하며, 스포츠카와 세단 등 다양한 차종을 보유하고 있습니다. 또한, BMW는 지속 가능한 모빌리티와 혁신적인 기술 개발에도 힘쓰고 있어, 최신 기술과 편의 시설을 갖춘 차량을 제공하고 있습니다. 전 세계적으로 많은 차량 소유자들에게 사랑받는 BMW는 뛰어난 품질과 안정성으로 유명하며, 고객들에게 혁신적인 운전 경험을 제공하는 것을 목표로 노력하고 있습니다.', response_metadata={'token_usage': {'completion_tokens': 341, 'prompt_tokens': 21, 'total_tokens': 362}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-00761646-4d8c-485e-b9f4-f5c5d7414ea2-0'), AIMessage(content='Audi는 독일의 자동차 제조사로, 고급차 시장에서 선두적인 위치를 차지하고 있는 브랜드입니다. 1909년에 설립된 Audi는 기술적인 혁신과 디자인적인 우수성으로 유명하며, 세계적으로 인정받는 자동차 브랜드 중 하나입니다. Audi의 자동차는 성능, 안전성, 편의성 등 다양한 측면에서 고객들에게 높은 만족을 제공하고 있으며, 다양한 모델 라인업을 보유하고 있어 다양한 고객층의 요구를 충족시킬 수 있습니다. Audi는 현재 Volkswagen 그룹의 일부로 소속되어 있으며, 전 세계적으로 많은 사랑을 받고 있는 브랜드입니다.', response_metadata={'to

## 3. 비동기

비동기의 경우 메서드 앞에 a를 붙이며, 호출시 await 키워드를 사용하는 것을 제외하면, 사용법은 동일.

## 4. 병렬 처리

RunnableParallel를 사용하여 작업을 병렬로 실행 가능.

In [19]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI(model_name="gpt-3.5-turbo")

chain1 = PromptTemplate.from_template("{country}의 수도는 어디야?") | model
chain2 = PromptTemplate.from_template("{country}의 대표적인 브랜드는 무엇이야?") | model

combined = RunnableParallel(capital=chain1, brand=chain2)

result = combined.invoke({"country": "대한민국"})

print(f"답변: {result}")

답변: {'capital': AIMessage(content='대한민국의 수도는 서울이야.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-242e4090-7403-4514-abdc-372017853691-0'), 'brand': AIMessage(content='대한민국의 대표적인 브랜드로는 삼성전자, 현대자동차, LG전자, 기아자동차 등이 있습니다. 이들은 국내외에서 매우 높은 인지도와 평판을 가지고 있는 대기업들입니다.', response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 33, 'total_tokens': 120}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-c3341104-4793-47b4-8d6a-451fef8dbbf0-0')}


배치 또한 병렬로 처리 가능

In [20]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI(model_name="gpt-3.5-turbo")

chain1 = PromptTemplate.from_template("{country}의 수도는 어디야?") | model
chain2 = PromptTemplate.from_template("{country}의 대표적인 브랜드는 무엇이야?") | model

combined = RunnableParallel(capital=chain1, brand=chain2)

result = combined.batch(
    [{"country": "대한민국"}, {"country": "미국"}],
	  config={"max_concurrency": 2}
)

print(f"답변: {result}")

답변: [{'capital': AIMessage(content='대한민국의 수도는 서울이야.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-91997652-683c-43fa-ae57-2206a263dfcc-0'), 'brand': AIMessage(content='대한민국의 대표적인 브랜드로는 삼성전자, 현대자동차, LG전자, 아모레퍼시픽, SK텔레콤 등이 있습니다. 이들 브랜드는 국내외에서 높은 인지도와 평판을 가지고 있으며 한국을 대표하는 글로벌 기업들입니다.', response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 33, 'total_tokens': 154}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-398f2de3-52ba-42b5-9b77-ae334d665f34-0')}, {'capital': AIMessage(content='미국의 수도는 워싱턴 D.C.입니다.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 19, 'total_tokens': 36}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None